In [1]:
import numpy as np
import tensorflow as tf

def pass_dataframe_to_tflite(df, tflite_model_path, string_lookup_table):
    # Load the TFLite model
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()

    # Get input and output details
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()

    # Prepare the input data
    input_data = df.copy()

    # Apply string lookup table to encode string values
    for i, column in enumerate(input_details):
        if column['dtype'] == np.object:
            lookup_table = string_lookup_table[i]
            input_data[column['name']] = input_data[column['name']].map(lookup_table)

    # Convert the input DataFrame to a NumPy array
    input_data = input_data.to_numpy(dtype=np.float32)

    # Set the input tensor
    interpreter.set_tensor(input_details[0]['index'], input_data)

    # Run the inference
    interpreter.invoke()

    # Get the output tensor
    output_data = interpreter.get_tensor(output_details[0]['index'])

    # Perform any post-processing on the output data if needed

    return output_data

2023-06-09 15:43:21.873408: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 15:43:21.911877: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-06-09 15:43:21.912468: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-09 15:43:22.545013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import pandas as pd

# Create a sample DataFrame
df = pd.DataFrame([['<0', 6.0, 'critical/other existing credit', 'radio/tv', 1169.0, 'no known savings', '>=7', 4.0, 'none', 4.0, 'real estate', 67.0, 'none', 'own', 2.0, 'skilled', 1.0, 'yes', 'yes','male','single']], columns=['checking_status', 'duration', 'credit_history', 'purpose',
           'credit_amount', 'savings_status', 'employment',
           'installment_commitment', 'other_parties', 'residence_since',
           'property_magnitude', 'age', 'other_payment_plans', 'housing',
           'existing_credits', 'job', 'num_dependents', 'own_telephone',
           'foreign_worker', 'sex', 'marriage'])

# Path to your TFLite model file
tflite_model_path = 'exportedMMobil.tflite'


In [3]:
string_lookup_tables = [
    np.array(['no checking', '<0', '0<=X<200', '>=200'], dtype=object),
    np.array(['existing paid', 'critical/other existing credit', 'delayed previously', 'all paid'], dtype=object),
    np.array(['radio/tv', 'new car', 'furniture/equipment', 'used car'], dtype=object),
    np.array(['<100', 'no known savings', '100<=X<500', '500<=X<1000'], dtype=object),
    np.array(['1<=X<4', '>=7', '4<=X<7', '<1'], dtype=object),
    np.array(['none', 'guarantor', 'co applicant'], dtype=object),
    np.array(['car', 'real estate', 'life insurance', 'no known property'], dtype=object),
    np.array(['none', 'bank', 'stores'], dtype=object),
    np.array(['own', 'rent', 'for free'], dtype=object),
    np.array(['skilled', 'unskilled resident', 'high qualif/self emp/mgmt', 'unemp/unskilled non res'], dtype=object),
    np.array(['none', 'yes'], dtype=object),
    np.array(['yes', 'no'], dtype=object),
    np.array(['male', 'female'], dtype=object),
    np.array(['single', 'div/dep/mar', 'mar/wid', 'div/sep'], dtype=object)
]

output = pass_dataframe_to_tflite(df, tflite_model_path, string_lookup_tables)

INFO: Created TensorFlow Lite delegate for select TF ops.
/tmp/ipykernel_7946/3523275565.py:18: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if column['dtype'] == np.object:
INFO: TfLiteFlexDelegate delegate: 14 nodes delegated out of 186 nodes with 1 partitions.

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


ValueError: could not convert string to float: '<0'

In [5]:

# Load the model
model = tf.keras.models.load_model('exportedModel')

In [6]:
print(model.summary())

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 checking_status (InputLayer)   [(None, 1)]          0           []                               
                                                                                                  
 credit_history (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 purpose (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 savings_status (InputLayer)    [(None, 1)]          0           []                               
                                                                                              

In [7]:
variables = model.get_weights()

In [8]:
print(variables)

[array([b'no checking', b'<0', b'0<=X<200', b'>=200'], dtype=object), array([b'existing paid', b'critical/other existing credit',
       b'delayed previously', b'all paid'], dtype=object), array([b'radio/tv', b'new car', b'furniture/equipment', b'used car'],
      dtype=object), array([b'<100', b'no known savings', b'100<=X<500', b'500<=X<1000'],
      dtype=object), array([b'1<=X<4', b'>=7', b'4<=X<7', b'<1'], dtype=object), array([b'none', b'guarantor', b'co applicant'], dtype=object), array([b'car', b'real estate', b'life insurance', b'no known property'],
      dtype=object), array([b'none', b'bank', b'stores'], dtype=object), array([b'own', b'rent', b'for free'], dtype=object), array([b'skilled', b'unskilled resident', b'high qualif/self emp/mgmt',
       b'unemp/unskilled non res'], dtype=object), array([b'none', b'yes'], dtype=object), array([b'yes', b'no'], dtype=object), array([b'male', b'female'], dtype=object), array([b'single', b'div/dep/mar', b'mar/wid', b'div/sep'], dtype